In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import h5py

import scipy.misc

# from utils import *

from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor


%matplotlib inline
tf.get_logger().setLevel("ERROR")

In [ ]:
np.random.seed(1)
tf.random.set_seed(2)

#### Identity Block
When the input activation has the same dimension as the output activation

In [ ]:
def identity_block(X, f, filters, initializer=random_uniform):
    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=1, strides=(1,1), padding="valid", kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation("relu")(X)

    X = Conv2D(filters=F2, kernel_size=f, strides=(1,1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation("relu")(X)

    X = Conv2D(filters=F3, kernel_size=1, strides=(1,1), padding="valid", kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)

    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)

    return X

### Convolutional Block

When the input and output dimensions don't match up

In [ ]:
def convolutional_block(X, f, filters, s=1, initializer=glorot_uniform):

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters = F1, kernel_size= 1, strides= (s,s), padding="valid", kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation("relu")(X)

    X = Conv2D(filters = F2, kernel_size= f, strides= (1,1), padding="same", kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation("relu")(X)

    X = Conv2D(filters = F3, kernel_size= 1, strides= (1,1), padding="valid", kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)

    X_shortcut = Conv2D(filters = F3, kernel_size= 1, strides= (s,s), padding="valid", kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=-1)(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)

    return X

### RESNET50 Model

In [ ]:
def ResNet50(input_shape=(64,64,3), classes=6):
    X_input = Input(input_shape)

    X = ZeroPadding2D((3,3))(X_input)

    X = Conv2D(64, kernel_size=7, strides=2, kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3), strides=(2,2))(X)

    X = convolutional_block(X, 3, [64,64,256], 1)
    X = identity_block(X, 3, [64,64,256])
    X = identity_block(X, 3, [64,64,256])

    X = convolutional_block(X, 3, [128,128,512], 2)
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])

    X = convolutional_block(X, 3, [256,256,1024], 2)
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])

    X = convolutional_block(X, 3, [512,512,2048], 2)
    X = identity_block(X, 3, [512,512,2048])
    X = identity_block(X, 3, [512,512,2048])

    X = AveragePooling2D(2)(X)
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", kernel_initializer=glorot_uniform(seed=0))(X)

    model = Model(inputs=X_input, outputs=X)

    return model

In [ ]:
model1 = ResNet50(input_shape = (64, 64, 3), classes = 6,)
# print(model.summary())

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00015)
model1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [ ]:
train_dataset = h5py.File("datasets/train_signs.h5")
train_set_x_orig = np.array(train_dataset["train_set_x"][:])
train_set_y_orig = np.array(train_dataset["train_set_y"][:]).reshape(1, -1)

test_dataset = h5py.File("datasets/test_signs.h5")
test_set_x_orig = np.array(test_dataset["test_set_x"][:])
test_set_y_orig = np.array(test_dataset["test_set_y"][:]).reshape(1, -1)

classes = np.array(test_dataset["list_classes"][:])

xtr = train_set_x_orig/255.
xtt = test_set_x_orig/255.

ytr = convert_to_one_hot(train_set_y_orig, 6).T
ytt = convert_to_one_hot(test_set_y_orig, 6).T

In [ ]:
model1.fit(xtr, ytr, epochs = 10, batch_size = 32)

In [ ]:
preds = model1.evaluate(xtt, ytt)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))